In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16,12)

In [ ]:
import sys ; sys.path += ['../']

from Pycode.Data_Aquisition import get_crypto_data
from Pycode.Features_Engineering import *
from Pycode.train_XGB import *

**TABLE OF CONTENT**
* [Data Pipeline](#data-bullet)
* [XGBoost](#xgb_run-bullet)

## Data Pipeline<a class="anchor" id="data-bullet"></a>

In [ ]:
# Params
n_batch_obs=4
currency = 'EUR'
exchange = 'Coinbase'
n_best = 400
n_val=60

# Get data from API
df = get_crypto_data(n_batch_obs=n_batch_obs,  currency=currency, exchange=exchange)

# Create the features dataset
feats_df = features_engineering(df)

# Create the target series
target, class_weights = target_engineering(df)

# Select Kbest features
kbest_selector, kbest_df = features_selection(feats_df, target, n_best)


# Prepare the data for XGB Trainng
df_train_scaled, df_val_scaled, target_train, target_val = XGB_Data_Preparation(kbest_df, target, n_val=n_val)



- EUR/BTC Prices from 1590890400 to 1598090400 
- EUR/BTC Prices from 1583690400 to 1590890400 
- EUR/BTC Prices from 1576490400 to 1583690400 
- EUR/BTC Prices from 1569290400 to 1576490400 
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8000 entries, 2019-09-24 02:00:00 to 2020-08-22 10:00:00
Data columns (total 6 columns):
close         8000 non-null float64
high          8000 non-null float64
low           8000 non-null float64
open          8000 non-null float64
volumefrom    8000 non-null float64
volumeto      8000 non-null float64
dtypes: float64(6)
memory usage: 437.5 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8000 entries, 2019-09-24 02:00:00 to 2020-08-22 10:00:00
Data columns (total 6 columns):
close         8000 non-null float64
high          8000 non-null float64
low           8000 non-null float64
open          8000 non-null float64
volumefrom    8000 non-null float64
volumeto      8000 non-null float64
dtypes: float64(6)
memory usage: 437.5 KB
None
<cl

../Pycode/Features_Engineering.py:66: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  feats_df[col + '.M{}{}H'.format(l, i)] = series.rolling("{}H".format(i)).apply(stat)
../Pycode/Features_Engineering.py:67: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  feats_df[col + '.M{}{}H.diff'.format(l, i)] = series.rolling("{}H".format(i)).apply(stat).diff()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8000 entries, 2019-09-24 02:00:00 to 2020-08-22 10:00:00
Columns: 858 entries, volumefrom to volumeto_mov24H.MSum24H.diff
dtypes: float64(858)
memory usage: 52.4 MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8000 entries, 2019-09-24 02:00:00 to 2020-08-22 10:00:00
Columns: 1716 entries, volumefrom to volumeto_mov24H.MSum24H.diff.Squared
dtypes: float64(1716)
memory usage: 104.8 MB


## Run the XGB pipeline<a class="anchor" id="xgb_run-bullet"></a>
### Open to Close Trading Signal

In [ ]:
# params
n_test = 200
n_tune = 2 # number of combinations to try during tuning


# Tuning for Close-Close Trading Signal
tuning_df = XGB_Tuning_Pipeline(df_train_scaled, df_val_scaled, target_train, target_val,
                                n_test=n_test, n_tune=n_tune)

# Get best params
best_params = tuning_df.iloc[0, 0]
print('Best parameters: {}'.format(best_params))

# Validation of the tuned model
df_test, df_backtest = XGB_Eval_Pipeline(best_params, 
                                         df_train_scaled, df_val_scaled, target_train, target_val, 
                                         n_test=n_test, _plot=True)


df_backtest